Computational Exercise
==
a.
--

In [1]:
using Optim
using DataFrames

In [2]:
df = readtable("hsdata.csv");
data = exp(convert(Matrix,df));
head(df)

,time,c,d,r
1,1,0.038243814,-0.139685811,-0.041037829
2,2,0.035488924,0.066637794,0.139808473
3,3,0.011135564,0.060121765,0.098999892
4,4,0.065275842,-0.015548595,0.089697058
5,5,0.044419714,-0.018336139,0.022571397
6,6,0.04769839,0.01983362,0.086112798


In [9]:
function g(w,z,theta) 
    return (theta[1] * w[1]^(-theta[2]) * w[2] - 1) .* collect(z[:])
end

function gn(w,z,theta)
    result = zeros(Float64, size(z)[2])
    for i in 1:size(w)[1]
        result .+= g(w[i,:],z[i,:],theta)
    end
    return result/size(w)[1]
end

#Wrapper creates a closure around the data provided
function Qn_wrapper(w,z)
    return theta -> (gn(w,z,theta)'*gn(w,z,theta))[1,1]
end

Qn_wrapper (generic function with 1 method)

In [24]:
w = data[2:end,[2,4]]
z = hcat(ones(size(data)[1]-1), data[1:end-1,4], 1 .* data[1:end-1,2])

Qn = Qn_wrapper(w,z)
optres = optimize(Qn, [1., 4.])

Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [1.0,4.0]
 * Minimizer: [0.9962444849312304,4.5251117866486315]
 * Minimum: 3.688057e-08
 * Iterations: 21
 * Convergence: true
   *  √(Σ(yᵢ-ȳ)²)/n < 1.0e-08: true
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 25


b.
--

In [25]:
theta = optres.minimum
What = zeros(Float64, size(z)[2], size(z)[2])
for i in 1:size(w)[1]
    What .+= g(w[i,:],z[i,:],theta)*g(w[i,:],z[i,:],theta)'
end
What = inv(What/size(z)[1])

function Qn_wrapper(w,z,What)
    return theta -> (gn(w,z,theta)'*What*gn(w,z,theta))[1,1]
end

QnWhat = Qn_wrapper(w,z,What)
optresWhat = optimize(QnWhat, theta)

Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [0.9962444849312304,4.5251117866486315]
 * Minimizer: [0.9790600035979021,3.418229100409705]
 * Minimum: 6.467554e-04
 * Iterations: 25
 * Convergence: true
   *  √(Σ(yᵢ-ȳ)²)/n < 1.0e-08: true
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 30


c.
--